Build a toy shapes dataset. Strongly inspired from [this implementation](https://github.com/matterport/Mask_RCNN/blob/cbff80f3e3f653a9eeee43d0d383a0385aba546b/samples/shapes/shapes.py).

In [4]:
%matplotlib inline

from pathlib import Path

import tensorflow as tf
import numpy as np
import pandas as pd
import tifffile

import sys
sys.path.append("../../../")
import maskflow

import utils

root_dir = Path("/home/hadim/.data/Neural_Network/Maskflow/Shapes")

data_dir = root_dir / "Data"
data_dir.mkdir(parents=True, exist_ok=True)

image_dir = data_dir / "Image"
image_dir.mkdir(parents=True, exist_ok=True)
mask_dir = data_dir / "Mask"
mask_dir.mkdir(parents=True, exist_ok=True)
class_dir = data_dir / "Class"
class_dir.mkdir(parents=True, exist_ok=True)

config = maskflow.load_config("config.yml")
class_names = config["CLASS_NAMES"]

# Copy config next to data folder

In [5]:
maskflow.save_config(config, root_dir / "config.yml")

# Generate the dataset

The dataset consists of a root folder containing 3 sub-folders:

- `Image/`: it contains the original images stored as TIF files. Shape is [H, W, C].
- `Mask/`: it contains the masks stored as TIF files. Shape is [N, W, H].
- `Class/`: Class ids as csv files.

In [6]:
height = config["IMAGE_MAX_DIM"]
width = config["IMAGE_MAX_DIM"]
count = 500
max_n_per_image = 3

for i in range(count):
    # Generate image specification
    bg_color, shapes = utils.random_image(height, width, max_n_per_image, class_names)
    
    # Generate the image
    image = utils.generate_image(bg_color, height, width, shapes)
    
    # Generate the mask
    mask, class_ids = utils.generate_mask(bg_color, height, width, shapes, class_names)
    
    # Save the iamge, mask and class ids
    basename = f"toy_{i:04d}" 
    image_path = image_dir / (basename + ".tif")
    mask_path = mask_dir / (basename + ".tif")
    class_ids_path = class_dir / (basename + ".csv")
    
    tifffile.imsave(str(image_path), image)
    tifffile.imsave(str(mask_path), mask)
    pd.Series(class_ids).to_csv(class_ids_path, index=False)